In [4]:
uri = "<URI_STRING>"

## Read data from MongoDB 

In [74]:
import pymongo

client = pymongo.MongoClient(uri)
db = client['sample_analytics']
coll = db['accounts']

documents_found = coll.find(filter={})

In [75]:
import json 
from bson import json_util

json_data = {}
for i, document in enumerate(documents_found):
    json_data = json.loads(json_util.dumps(document))

print(json_data)

{'_id': {'$oid': '5ca4bbc7a2dd94ee58162a60'}, 'limit': 10000, 'products': ['Commodity', 'InvestmentStock'], 'account id': 291224}


## Create Spark session

### Convert JSON to Parquet using parallelize

In [77]:
import shutil 
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("JsonToParquetConverter").getOrCreate()
# Read nested JSON data
jsonData = spark.read.json(spark.sparkContext.parallelize([json_data]))
# Coalesce the data into a single partition
jsonData = jsonData.coalesce(1)
# Write JSON data to Parquet format, overwriting if already exists
jsonData.write.mode("overwrite").parquet("test.parquet")

#### Read paruqet file Schema and Data 

In [78]:
parquetData = spark.read.parquet("test.parquet")
# Example: Show the schema and first few rows of the Parquet data
parquetData.printSchema()
parquetData.show(5)

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- account id: long (nullable = true)
 |-- limit: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+----------+-----+--------------------+
|                 _id|account id|limit|            products|
+--------------------+----------+-----+--------------------+
|{5ca4bbc7a2dd94ee...|    291224|10000|[Commodity, Inves...|
+--------------------+----------+-----+--------------------+



#### Stop spark job

In [ ]:
# Stop the SparkSession
spark.stop()